In [2]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import spacy

RS = 10424
print("Started")
np.random.seed(RS)
input_folder = './input/'
pd.options.display.max_colwidth = 200

Started


In [3]:
nlp = spacy.load('en_core_web_md')

In [5]:
print('Sanity test:')
#doc = nlp(u'I am a good programmer!')
doc = nlp(u'how would you review the site Waveclues ?')
print([t.ent_type_ for t in doc])

df_train = pd.read_csv(input_folder + 'train.csv')
y_train = df_train['is_duplicate'].values
#df_test  = pd.read_csv(input_folder + 'test.csv')

#print("Original data: X_train: {}, X_test: {}".format(df_train.shape, df_test.shape))

Sanity test:
['', '', '', '', '', '', '', '']


In [4]:
x_train = pd.DataFrame.from_csv('./datasets/why_jaccard_x_intersect_train_extended.csv').reset_index(drop=['index'])
#x_test = pd.DataFrame.from_csv('./datasets/why_jaccard_x_intersect_test.csv').reset_index(drop=['index'])
y_train = df_train['is_duplicate'].values

NameError: name 'df_train' is not defined

In [6]:
qid_dict = {}

for i,series in df_train.iterrows():
    if series['qid1'] not in qid_dict:
        qid_dict[series['qid1']] = series['question1']
    if series['qid2'] not in qid_dict:
        qid_dict[series['qid2']] = series['question2']

In [13]:
import pickle

#pickle.dump(qid_dict, open('./datasets/qid_dict.pkl', 'wb'))
q_dict = pickle.load( open('./datasets/qid_dict.pkl', 'rb') )        

In [11]:
SPECIAL_TOKENS = {
    'quoted': 'quoted_item',
    'non-ascii': 'non_ascii_word',
    'undefined': 'something'
}

In [9]:
import re

def clean_string(text):
    
    def pad_str(s):
        return ' '+s+' '
    
    # Empty question
    
    if type(text) != str or text=='':
        return ''
    
    # preventing first and last word being ignored by regex
    # and convert first word in question to lower case
    
    text = ' ' + text[0].lower() + text[1:] + ' '
    
    # replace all first char after either [.!?)"'] with lowercase
    # don't mind if we lowered a proper noun, it won't be a big problem
    
    def lower_first_char(pattern):
        matched_string = pattern.group(0)
        return matched_string[:-1] + matched_string[-1].lower()
    
    text = re.sub("(?<=[\.\?\)\!\'\"])[\s]*.",lower_first_char , text)
    
    # Replace weird chars in text
    
    text = re.sub("’", "'", text) # special single quote
    text = re.sub("`", "'", text) # special single quote
    text = re.sub("“", '"', text) # special double quote
    text = re.sub("？", "?", text) 
    text = re.sub("…", " ", text) 
    text = re.sub("é", "e", text) 
    
    # Clean shorthands
    
    text = re.sub("\'s", " ", text) # we have cases like "Sam is" or "Sam's" (i.e. his) these two cases aren't separable, I choose to compromise are kill "'s" directly
    text = re.sub(" whats ", " what is ", text, flags=re.IGNORECASE)
    text = re.sub("\'ve", " have ", text)
    text = re.sub("can't", "can not", text)
    text = re.sub("n't", " not ", text)
    text = re.sub("i'm", "i am", text, flags=re.IGNORECASE)
    text = re.sub("\'re", " are ", text)
    text = re.sub("\'d", " would ", text)
    text = re.sub("\'ll", " will ", text)
    text = re.sub("e\.g\.", " eg ", text, flags=re.IGNORECASE)
    text = re.sub("b\.g\.", " bg ", text, flags=re.IGNORECASE)
    text = re.sub(r"(\W|^)([0-9]+)[kK](\W|$)", r"\1\g<2>000\3", text) # better regex provided by @armamut
    text = re.sub("e-mail", " email ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?U\.S\.A\.", " America ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?United State(s)?", " America ", text, flags=re.IGNORECASE)
    text = re.sub("\(s\)", " ", text, flags=re.IGNORECASE)
    text = re.sub("[c-fC-F]\:\/", " disk ", text)
    
    # replace the float numbers with a random number, it will be parsed as number afterward, and also been replaced with word "number"
    
    text = re.sub('[0-9]+\.[0-9]+', " 87 ", text)
    
    # remove comma between numbers, i.e. 15,000 -> 15000
    
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
    
#     # all numbers should separate from words, this is too aggressive
    
#     def pad_number(pattern):
#         matched_string = pattern.group(0)
#         return pad_str(matched_string)
#     text = re.sub('[0-9]+', pad_number, text)
    
    # add padding to punctuations and special chars, we still need them later
    
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)
    
    def pad_pattern(pattern):
        matched_string = pattern.group(0)
        return pad_str(matched_string)
    text = re.sub('[\!\?\@\^\+\*\/\,\~\|\`\=\:\;\.\#\\\]', pad_pattern, text) 
        
    text = re.sub('[^\x00-\x7F]+', pad_str(SPECIAL_TOKENS['non-ascii']), text) # replace non-ascii word with special word
    
    # indian dollar
    
    text = re.sub("(?<=[0-9])rs ", " rs ", text, flags=re.IGNORECASE)
    text = re.sub(" rs(?=[0-9])", " rs ", text, flags=re.IGNORECASE)
    
    # clean text rules get from : https://www.kaggle.com/currie32/the-importance-of-cleaning-text
    
    text = re.sub(r" (the[\s]+|The[\s]+)?US(A)? ", " America ", text)
    text = re.sub(r" UK ", " England ", text, flags=re.IGNORECASE)
    text = re.sub(r" india ", " India ", text)
    text = re.sub(r" switzerland ", " Switzerland ", text)
    text = re.sub(r" china ", " China ", text)
    text = re.sub(r" chinese ", " Chinese ", text) 
    text = re.sub(r" imrovement ", " improvement ", text, flags=re.IGNORECASE)
    text = re.sub(r" intially ", " initially ", text, flags=re.IGNORECASE)
    text = re.sub(r" quora ", " Quora ", text, flags=re.IGNORECASE)
    text = re.sub(r" dms ", " direct messages ", text, flags=re.IGNORECASE)  
    text = re.sub(r" demonitization ", " demonetization ", text, flags=re.IGNORECASE) 
    text = re.sub(r" actived ", " active ", text, flags=re.IGNORECASE)
    text = re.sub(r" kms ", " kilometers ", text, flags=re.IGNORECASE)
    text = re.sub(r" cs ", " computer science ", text, flags=re.IGNORECASE) 
    text = re.sub(r" upvote", " up vote", text, flags=re.IGNORECASE)
    text = re.sub(r" iPhone ", " phone ", text, flags=re.IGNORECASE)
    text = re.sub(r" \0rs ", " rs ", text, flags=re.IGNORECASE)
    text = re.sub(r" calender ", " calendar ", text, flags=re.IGNORECASE)
    text = re.sub(r" ios ", " operating system ", text, flags=re.IGNORECASE)
    text = re.sub(r" gps ", " GPS ", text, flags=re.IGNORECASE)
    text = re.sub(r" gst ", " GST ", text, flags=re.IGNORECASE)
    text = re.sub(r" programing ", " programming ", text, flags=re.IGNORECASE)
    text = re.sub(r" bestfriend ", " best friend ", text, flags=re.IGNORECASE)
    text = re.sub(r" dna ", " DNA ", text, flags=re.IGNORECASE)
    text = re.sub(r" III ", " 3 ", text)
    text = re.sub(r" banglore ", " Banglore ", text, flags=re.IGNORECASE)
    text = re.sub(r" J K ", " JK ", text, flags=re.IGNORECASE)
    text = re.sub(r" J\.K\. ", " JK ", text, flags=re.IGNORECASE)
    
    # typos identified with my eyes
    
    text = re.sub(r" quikly ", " quickly ", text)
    text = re.sub(r" unseccessful ", " unsuccessful ", text)
    text = re.sub(r" demoniti[\S]+ ", " demonetization ", text, flags=re.IGNORECASE)
    text = re.sub(r" demoneti[\S]+ ", " demonetization ", text, flags=re.IGNORECASE)  
    text = re.sub(r" addmision ", " admission ", text)
    text = re.sub(r" insititute ", " institute ", text)
    text = re.sub(r" connectionn ", " connection ", text)
    text = re.sub(r" permantley ", " permanently ", text)
    text = re.sub(r" sylabus ", " syllabus ", text)
    text = re.sub(r" sequrity ", " security ", text)
    text = re.sub(r" undergraduation ", " undergraduate ", text) # not typo, but GloVe can't find it
    text = re.sub(r"(?=[a-zA-Z])ig ", "ing ", text)
    text = re.sub(r" latop", " laptop", text)
    text = re.sub(r" programmning ", " programming ", text)  
    text = re.sub(r" begineer ", " beginner ", text)  
    text = re.sub(r" qoura ", " Quora ", text)
    text = re.sub(r" wtiter ", " writer ", text)  
    text = re.sub(r" litrate ", " literate ", text)  

      
    # for words like A-B-C-D or "A B C D", 
    # if A,B,C,D individuaally has vector in glove:
    #     it can be treat as separate words
    # else:
    #     replace it as a special word, A_B_C_D is enough, we'll deal with that word later
    #
    # Testcase: 'a 3-year-old 4 -tier car'
    
    def dash_dealer(pattern):
        matched_string = pattern.group(0)
        splited = matched_string.split('-')
        splited = [sp.strip() for sp in splited if sp!=' ' and sp!='']
        joined = ' '.join(splited)
        parsed = nlp(joined)
        for token in parsed:
            # if one of the token is not common word, then join the word into one single word
            if not token.has_vector or token.text in SPECIAL_TOKENS.values():
                return '_'.join(splited)
        # if all tokens are common words, then split them
        return joined

    text = re.sub("[a-zA-Z0-9\-]*-[a-zA-Z0-9\-]*", dash_dealer, text)
    
    # try to see if sentence between quotes is meaningful
    # rule:
    #     if exist at least one word is "not number" and "length longer than 2" and "it can be identified by SpaCy":
    #         then consider the string is meaningful
    #     else:
    #         replace the string with a special word, i.e. quoted_item
    # Testcase:
    # i am a good (programmer)      -> i am a good programmer
    # i am a good (programmererer)  -> i am a good quoted_item
    # i am "i am a"                 -> i am quoted_item
    # i am "i am a programmer"      -> i am i am a programmer
    # i am "i am a programmererer"  -> i am quoted_item
    
    def quoted_string_parser(pattern):
        string = pattern.group(0)
        parsed = nlp(string[1:-1])
        is_meaningful = False
        for token in parsed:
            # if one of the token is meaningful, we'll consider the full string is meaningful
            if len(token.text)>2 and not token.text.isdigit() and token.has_vector:
                is_meaningful = True
            elif token.text in SPECIAL_TOKENS.values():
                is_meaningful = True
            
        if is_meaningful:
            return string
        else:
            return pad_str(string[0]) + SPECIAL_TOKENS['quoted'] + pad_str(string[-1])

    text = re.sub('\".*\"', quoted_string_parser, text)
    text = re.sub("\'.*\'", quoted_string_parser, text)
    text = re.sub("\(.*\)", quoted_string_parser, text)
    text = re.sub("\[.*\]", quoted_string_parser, text)
    text = re.sub("\{.*\}", quoted_string_parser, text)
    text = re.sub("\<.*\>", quoted_string_parser, text)

    text = re.sub('[\(\)\[\]\{\}\<\>\'\"]', pad_pattern, text) 
    
    # the single 's' in this stage is 99% of not clean text, just kill it
    text = re.sub(' s ', " ", text)
    
    # reduce extra spaces into single spaces
    text = re.sub('[\s]+', " ", text)
    text = text.strip()
    
    return text

In [14]:
# %%time

processed = []

for k in qid_dict:
    if k%100000==0:
        print('Processed {} out of {}'.format(k,len(qid_dict)))
    
    temp = clean_string(qid_dict[k])
    
    if qid_dict[k] != temp:
        processed.append(k)
        qid_dict[k] = clean_string(qid_dict[k])

Processed 100000 out of 537933
Processed 200000 out of 537933
Processed 300000 out of 537933
Processed 400000 out of 537933
Processed 500000 out of 537933


In [21]:
#set(map(lambda k: qid_dict[k], processed))
#keys = set(qid_dict.keys())
pkeys = set(processed)

unprocessed = [qid for qid in qid_dict.keys() if qid not in pkeys]

In [24]:
#len(unprocessed)
for k in unprocessed[:1000]:
    print(qid_dict[k])

what is the step by step guide to invest in share market in India ?
what is the step by step guide to invest in share market ?
what is the story of Kohinoor ( Koh i Noor ) diamond ?
what would happen if the Indian government stole the Kohinoor ( Koh i Noor ) diamond back ?
how can I increase the speed of my internet connection while using a VPN ?
how can Internet speed be increased by hacking through DNS ?
why am I mentally very lonely ? how can I solve it ?
find the remainder when [ math ] 23 ^ { quoted_item } [ / math ] is divided by 2423 ?
which one dissolve in water quickly sugar , salt , methane and carbon di oxide ?
which fish would survive in salt water ?
astrology : I am a Capricorn Sun Cap moon and cap rising . . . what does that say about me ?
i am a triple Capricorn ( Sun , Moon and ascendant in Capricorn ) what does this say about me ?
should I buy tiago ?
what keeps childern active and far from phone and video games ?
how can I be a good geologist ?
what should I do to be 

In [ ]:
# %%time

spacy_obj_dict = {}
total_len = len(qid_dict)
for i,k in enumerate(qid_dict):
    if i%50000==0:
        print('Processed {} out of {}'.format(i,total_len))
        
    # some questions are null
    if type(qid_dict[k])!= str:
        qid_dict[k] = ''
    
    spacy_obj_dict[k] = nlp(qid_dict[k]

net_dict = {}

for i, q in df_train.iterrows():
    if series['qid1'] not in qid_dict:
        qid_dict[series['qid1']] = nlp(series['question1']).M
    if series['qid2'] not in qid_dict:
        qid_dict[series['qid2']] = series['question2']    


In [ ]:
doc1 = nlp(u'London is a big city in the United Kingdom.')
doc2 = nlp(u'I\'m a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?')
print(doc[0].text, doc[0].ent_iob, doc[0].ent_type_)

In [ ]:
#pos_train = df_train[y_train == 1]

docs = pos_train.loc[1:1000,:].apply(lambda q: (nlp(q['question1']), nlp(q['question2'])) , axis=1)
#pos_train.head(n=1000)
#doc1.similarity(doc2)

In [ ]:
vote_dict = {}
word_ent_type_dict = {}
word_ent_type_second_dict = {}

# construct vote dictionary

for qid in spacy_obj_dict:
    
    if qid%100000==0:
        print('Processing {} / {}'.format(qid,len(spacy_obj_dict)))
    
    for token in spacy_obj_dict[qid]:
            
        if token.lower_ not in vote_dict:
            vote_dict[token.lower_] = {}

        if token.ent_type_ not in vote_dict[token.lower_]:
            vote_dict[token.lower_][token.ent_type_] = 0
        
        # if the token has_vector is True, maybe we shouldn't record its 
        
        vote_dict[token.lower_][token.ent_type_] += 1 # TODO: not sure if storing in lowercase form is safe ?

        
# vote for what should the type be
    
for key in vote_dict:
    
    # non-type has lower priority
    if '' in vote_dict[key]:
        vote_dict[key][''] = vote_dict[key][''] - 0.1

    ents = list(vote_dict[key].keys())
    bi_list = [
        ents,
        [vote_dict[key][ent] for ent in ents]
    ]
    
    # if several ent_type_ have same count, just let it go, making them share same ent_type_ is enough
    # TODO: if have time, can design a better metric to deal with second graded type
    sorted_idx = np.argsort(bi_list[1])
    
    if sorted_idx.shape[0]>1:
        best_idx = sorted_idx[-1]
        second_idx = sorted_idx[-2]
        word_ent_type_dict[key] = bi_list[0][best_idx]
        if bi_list[1][second_idx]>threshold_of_second_type:
            word_ent_type_second_dict[key] = bi_list[0][second_idx]
    else:
        best_idx = sorted_idx[-1]
        word_ent_type_dict[key] = bi_list[0][best_idx]

In [ ]:
# APIs to access entity type

def token_type_lookup(token, report_detail=False):
    
    if type(token)==str:
        token = nlp(token)[0]
        
    key = token.lower_
    
    try:
        if report_detail:
            print(ENTITY_ENUM[word_ent_type_dict[key]], ' <= ', {ENTITY_ENUM[ent_t] : vote_dict[key][ent_t] for ent_t in vote_dict[key]} )

        return word_ent_type_dict[key]
    
    except KeyError:
        return ''

def is_token_has_second_type(token):
    
    if type(token)==str:
        token = nlp(token)[0]
        
    key = token.lower_
    
    try:
        return key in word_ent_type_second_dict
    except KeyError:
        return False

def token_second_type_lookup(token, report_detail=False):
    
    if type(token)==str:
        token = nlp(token)[0]
        
    key = token.lower_
    
    try:
        if report_detail:
            print(ENTITY_ENUM[word_ent_type_second_dict[key]], ' <= ', {ENTITY_ENUM[ent_t] : vote_dict[key][ent_t] for ent_t in vote_dict[key]} )

        return word_ent_type_second_dict[key]
    except KeyError:
        return ''

In [ ]:
testcases = [
    'ms',
    'taiwanese',
    'google',
    'tensorflow',
    'mac'
]

for testcase in testcases:
    print(testcase)
    token_type_lookup(testcase, report_detail=True)
    print()

In [ ]:
i = 0

for doc in docs:
    print(doc[0], [(e.label_, e.text) for e in doc[0].ents])
    print(doc[1], [(e.label_, e.text) for e in doc[1].ents])
    i += 1
    
    if i > 1000:
        break

In [ ]:
#x_train.head()
#df_train.loc[1,['question1', 'question2']]
doc1 = nlp(df_train.loc[2,'question1'])
doc2 = nlp(df_train.loc[2,'question2'])

#df_train.loc[2,:]

In [ ]:
#qid_dict